In [34]:
import calliope
import warnings
import Graphs as gg

In [35]:
warnings.filterwarnings("ignore")

try:
    calliope.set_log_level('INFO')
except:
    calliope.set_log_verbosity('INFO')
model = calliope.Model('model.yaml')
model.run()

[2020-08-03 12:28:14] INFO     Model: initialising
[2020-08-03 12:28:15] INFO     Model: preprocessing stage 1 (model_run)
[2020-08-03 12:28:16] INFO     NumExpr defaulting to 4 threads.
[2020-08-03 12:28:16] INFO     Model: preprocessing stage 2 (model_data)
[2020-08-03 12:28:17] INFO     Model: preprocessing complete
[2020-08-03 12:28:17] INFO     Backend: starting model run
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lifetime) to be 'Any', you can suppress
    this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for Param objects is 'Any'.  However,
we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Param (lifetime) to be 'Any', you can
suppress this warning by explicitly specifying 'within=Any' to the
Param constructor.  (depreca

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (resource_unit) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for Param objects is 'Any'.  However,
we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Param (resource_unit) to be 'Any',
you can suppress this warning by explicitly specifying 'within=Any' to
the Param constructor.  (deprecated in 5.6.9, will be removed in 6.0)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_remotes) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECAT

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_remotes) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for Param objects is 'Any'.  However,
we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Param (lookup_remotes) to be 'Any',
you can suppress this warning by explicitly specifying 'within=Any' to
the Param constructor.  (deprecated in 5.6.9, will be removed in 6.0)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_remotes) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPREC

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (colors) to be 'Any', you can suppress
    this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for Param objects is 'Any'.  However,
we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Param (colors) to be 'Any', you can
suppress this warning by explicitly specifying 'within=Any' to the
Param constructor.  (deprecated in 5.6.9, will be removed in 6.0)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (colors) to be 'Any', you can suppress
    this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (inheritance) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for Param objects is 'Any'.  However,
we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Param (inheritance) to be 'Any', you
can suppress this warning by explicitly specifying 'within=Any' to the
Param constructor.  (deprecated in 5.6.9, will be removed in 6.0)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (inheritance) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (names) to be 'Any', you can suppress this
    warning by explicitly specifying 'within=Any' to the Param constructor.
    (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for Param objects is 'Any'.  However,
we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Param (names) to be 'Any', you can
suppress this warning by explicitly specifying 'within=Any' to the
Param constructor.  (deprecated in 5.6.9, will be removed in 6.0)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (names) to be 'Any', you can suppress this
    warning by explicitly specifying 'within=Any' to the Param constructor.
    (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for 

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_loc_carriers) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for Param objects is 'Any'.  However,
we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Param (lookup_loc_carriers) to be
'Any', you can suppress this warning by explicitly specifying
'within=Any' to the Param constructor.  (deprecated in 5.6.9, will be
removed in 6.0)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_loc_carriers) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] 

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_loc_techs) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for Param objects is 'Any'.  However,
we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Param (lookup_loc_techs) to be 'Any',
you can suppress this warning by explicitly specifying 'within=Any' to
the Param constructor.  (deprecated in 5.6.9, will be removed in 6.0)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_loc_techs) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_loc_techs) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for Param objects is 'Any'.  However,
we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Param (lookup_loc_techs) to be 'Any',
you can suppress this warning by explicitly specifying 'within=Any' to
the Param constructor.  (deprecated in 5.6.9, will be removed in 6.0)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_loc_techs) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_loc_techs) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  DEPRECATED: The default domain for Param objects is 'Any'.  However,
we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Param (lookup_loc_techs) to be 'Any',
you can suppress this warning by explicitly specifying 'within=Any' to
the Param constructor.  (deprecated in 5.6.9, will be removed in 6.0)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (lookup_loc_techs) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0)
[2020-08-03 12:28:19] WARNING  

In [1]:
import pandas as pd

In [24]:
Dem_Tech = pd.read_excel('Graph_inputs.xlsx',sheet_name = 'Consumption Techs',index_col=[0],header=[0])
Nodes    = pd.read_excel('Graph_inputs.xlsx',sheet_name = 'Nodes',index_col=[0],header=[0])
Pr_Techs = pd.read_excel('Graph_inputs.xlsx',sheet_name = 'Production Techs',index_col = [0],header=[0])
colname  = pd.read_excel('Graph_inputs.xlsx',sheet_name = 'Col_Name',index_col = [0],header =[0])
imp_exp  = pd.read_excel('Graph_inputs.xlsx',sheet_name = 'imp-exp',index_col = [0],header =[0])
Trans    = pd.read_excel('Graph_inputs.xlsx',sheet_name = 'Transmission',index_col = [0],header =[0])
date     = pd.read_excel('Graph_inputs.xlsx',sheet_name = 'Date',index_col = [0],header =[0])

In [30]:
co_techs = Dem_Tech['Tech'].to_list()
carr     = Dem_Tech['Carrier'].to_list()
nodes    = Nodes['Location'].to_list()
pr_techs = Pr_Techs['Tech'].to_list()
colors   = colname['Color']
names    = colname['Name']
imps     = imp_exp['Imports'].to_list()
exps     = imp_exp['Exports'].to_list()
tr_tech  = Trans['Tech'].to_list()
start    = date['Value'][0]
end      = date['Value'][1]

In [ ]:
# How to build the transmissions

In [57]:
model.inputs.loc_techs

<xarray.DataArray 'loc_techs' (loc_techs: 35)>
array(['CSTR::hvdc_132:MTKR', 'MTKR::S_hydro_pp', 'NBOR::HFO_pp',
       'NBOR::on_wind_pp', 'NBOR::L_hydro_pp', 'NBOR::GT_pp',
       'MTKR::hvdc_132:CSTR', 'CSTR::demand_power', 'NBOR::hvdc_220:WSTR',
       'MTKR::hvdc_132:WSTR', 'NBOR::free_transmission:NBOR',
       'WSTR::hvdc_132:MTKR', 'CSTR::free_transmission:CSTR',
       'WSTR::L_hydro_pp', 'NBOR::hvdc_132:WSTR', 'WSTR::demand_power',
       'WSTR::free_transmission:WSTR', 'MTKR::hvdc_132:NBOR',
       'WSTR::hvdc_132:NBOR', 'WSTR::hvdc_220:NBOR', 'CSTR::hvdc_220:MTKR',
       'MTKR::L_hydro_pp', 'NBOR::hvdc_132:MTKR', 'WSTR::geo_pp',
       'WSTR::bio_pp', 'MTKR::demand_power', 'MTKR::free_transmission:MTKR',
       'MTKR::hvdc_220:NBOR', 'NBOR::hvdc_220:MTKR', 'MTKR::hvdc_220:CSTR',
       'NBOR::hvdc_132:CSTR', 'NBOR::demand_power', 'CSTR::HFO_pp',
       'WSTR::S_hydro_pp', 'CSTR::hvdc_132:NBOR'], dtype=object)
Coordinates:
  * loc_techs  (loc_techs) object 'CSTR::hvdc_132:MTKR' ... 'CSTR::hvdc_132:NBOR'

In [78]:
HFO_4_regions = model.get_formatted_array('carrier_prod').loc[{'techs':['HFO_pp','S_hydro_pp'],'carriers':'power','locs':['WSTR','MTKR','NBOR','CSTR']}]


In [79]:
HFO_4_regions

<xarray.DataArray 'carrier_prod' (locs: 4, techs: 2, timesteps: 48)>
array([[[            nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan],
        [   797.5416667 ,    797.5416667 ,    797.5416667 ,
            797.5416667 ,    797.5416667 ,    797.5416667 ,
            797.5416667 ,    797.5416667 ,    797.5416667 ,
            797.5416667 ,    797.5416667 ,    797.5416667 ,
            797.5416667 ,    797.5416667 ,    797.5416667 ,
            797.5416667 ,    797.5416667 ,    797.5416667 ,
            797.5416667 ,    797.5416667 ,    797.5416667 ,
            797.5416667 ,    797.5416667 ,    797.5416667 ,
            837.5416667 ,    837.5416667 ,    837.5416667 ,
              0.        ,      0.        ,    837.5416667 ,
            837.5416667 ,    837.5416667 ,    837.5416667 ,
            837.5416667 ,    837.5416667 ,    837.5416667 ,
            837.5416667 ,    837.5416667 ,    837.5416667 ,
            837.5416667 ,    837.5416667 ,    837.5416667 ,
            837.5416667 ,    837.5416667 ,    837.5416667 ,
            837.5416667 ,    837.5416667 ,    837.5416667 ]],

       [[            nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan,
                     nan,             nan,             nan],
        [  8268.208333  ,   8266.208333  ,   8270.208333  ,
           8269.208333  ,   8272.208333  ,   8271.208333  ,
           8259.208333  ,   8258.208333  ,   8257.208333  ,
           8257.208333  ,   8257.208333  ,   8253.208333  ,
           8297.208333  ,   8297.208333  ,   8297.208333  ,
           8296.208333  ,   8295.208333  ,   8293.208333  ,
           8293.208333  ,   8265.208333  ,   8267.208333  ,
           8267.208333  ,   8267.208333  ,   8267.208333  ,
           7982.        ,   7982.        ,   7983.        ,
           7981.        ,   7980.        ,   7980.        ,
           7985.        ,   7994.        ,   7995.        ,
           7996.        ,   7996.        ,   7996.        ,
           7978.        ,   7976.        ,   7974.        ,
           7974.        ,   7972.        ,   7972.        ,
           7976.        ,   7976.        ,   7975.        ,
           7976.        ,   7976.        ,   7976.        ]],

       [[     0.        ,      0.        ,      0.        ,
              0.       

In [52]:
a= pd.concat([HFO_4_regions,HFO_4_regions],axis=1)

In [53]:
a

,0,1
timesteps,,
2015-01-01 00:00:00,0.000000,0.000000
2015-01-01 01:00:00,0.000000,0.000000
2015-01-01 02:00:00,0.000000,0.000000
2015-01-01 03:00:00,0.000000,0.000000
2015-01-01 04:00:00,0.000000,0.000000
2015-01-01 05:00:00,0.000000,0.000000
2015-01-01 06:00:00,0.000000,0.000000
2015-01-01 07:00:00,0.000000,0.000000
2015-01-01 08:00:00,0.000000,0.000000
